In [1]:
import pandas as pd
import json

In [ ]:
pip install python-twitter

In [2]:
consumer_key = "GseoXrC7Hie86zlpGcHBH3e6u"
consumer_secret = "hiVX0iHIWROzXgkKp3KahO8OydWZxFrdu4hek46odFh7QCL0R1"
access_token_key = "1064702341033730048-isbdsHb4poViCcM2B9bDlx1dHYe4iD"
access_token_secret = "zhVkJtwqNqSP3GuXTWjWdqYofRmSaX8OGzBh0MqdvG3It"

In [3]:
import twitter

api = twitter.Api(consumer_key = consumer_key,
                  consumer_secret = consumer_secret,
                  access_token_key = access_token_key,
                  access_token_secret = access_token_secret,
                  tweet_mode = "extended")

In [4]:
twitter = pd.read_excel("twitter_2011.xlsx")
twitter.head(3)

,EXEC_FULLNAME,CFOANN,CONAME,CEOANN,AGE,GVKEY,YEAR,BECAMECEO,GENDER,CUSIP,SPCODE,CEO_TWITTER,FIRM_TWITTER
0,Abraham N. Reichental,NaN,3D SYSTEMS CORP,CEO,54.0,14898,2011,2003-09-19,MALE,88554D20,SM,na,3dsystems
1,Abraham N. Reichental,NaN,3D SYSTEMS CORP,CEO,55.0,14898,2012,2003-09-19,MALE,88554D20,SM,na,3dsystems
2,Abraham N. Reichental,NaN,3D SYSTEMS CORP,CEO,56.0,14898,2013,2003-09-19,MALE,88554D20,SM,na,3dsystems


In [5]:
# remove all duplicate values
twitter.drop_duplicates(subset = "CEO_TWITTER", inplace = True)

# remove all "na" values
twitter = twitter[twitter["CEO_TWITTER"] != "na"]
twitter.head(3)

,EXEC_FULLNAME,CFOANN,CONAME,CEOANN,AGE,GVKEY,YEAR,BECAMECEO,GENDER,CUSIP,SPCODE,CEO_TWITTER,FIRM_TWITTER
18,Vikram Verma,NaN,8X8 INC,CEO,49.0,65021,2013,2013-09-09,MALE,28291410,SM,vikverma8x8,8x8
31,David P. Storch,NaN,AAR CORP,CEO,59.0,1004,2011,1996-10-09,MALE,00036110,SM,storchy1,aarcorp
124,Robert A. Kotick,NaN,ACTIVISION BLIZZARD INC,CEO,48.0,180405,2011,1991-02-01,MALE,00507V10,SP,bobbykotick,atvi_ab


In [6]:
ceo_twitter = list(twitter["CEO_TWITTER"])
print(f"{ceo_twitter[:10]} \n\nTotal Twitter Accounts: {len(ceo_twitter)}")

['vikverma8x8', 'storchy1', 'bobbykotick', 'lisawwardell', 'RoryPRead', 'lisasu', 'mikeaecom', 'DrakeEileen', 'AndresGluski', 'RameshAgilysys'] 

Total Twitter Accounts: 219


In [7]:
def get_timeline(api=None, acct=None):
    
    try:
        timeline = api.GetUserTimeline(screen_name = acct, count = 200)
        return timeline
    
    except Exception:
        print(f"Invalid Account: {acct}")
        pass

In [8]:
twitter_timeline = {}

for acct in ceo_twitter:
    timeline = get_timeline(api, acct)
    
    if bool(timeline) == True:
        twitter_timeline[acct] = timeline
        
    else:
        continue

Invalid Account: vikverma8x8
Invalid Account: mikeaecom
Invalid Account: susansalka
Invalid Account: CEOlarrydeshon
Invalid Account: janzijd
Invalid Account: boeingceo
Invalid Account: MichaelKWirth1
Invalid Account: leighfox
Invalid Account: markbtempleton
Invalid Account: jamesquincey
Invalid Account: Donahoe_John
Invalid Account: richboehne
Invalid Account: emeyercord
Invalid Account: Stuartessig1
Invalid Account: thomas_peterffy
Invalid Account: arnesorensons
Invalid Account: SteveEasterbrk
Invalid Account: HarryatNIC
Invalid Account: danrdimicco
Invalid Account: DhannahDave
Invalid Account: BlakeDMoret
Invalid Account: ScottSmithSonic
Invalid Account: SYKESCEO
Invalid Account: CalvinKnowlton
Invalid Account: strausszelnick
Invalid Account: Urstadt
Invalid Account: RickBelluzzo
Invalid Account: YeagyBrent
Invalid Account: MattMaddox_
Invalid Account: gretmcclain


In [9]:
twitter_timeline["adenatfriedman"]

[Status(ID=1394287953023750144, ScreenName=adenatfriedman, Created=Mon May 17 13:45:27 +0000 2021, Text='Markets are changing, and fast. @Nasdaq is taking steps to future-proof our infrastructure, build new marketplaces, and ensure a fair and level playing field for all market participants. Here are my thoughts in @barronsonline \nhttps://t.co/9g9zgXOyqf'),
 Status(ID=1388197431330889729, ScreenName=adenatfriedman, Created=Fri Apr 30 18:23:53 +0000 2021, Text='Honeywell’s technologies make the world smarter, safer, and more sustainable. We are delighted to welcome @honeywell to our family of innovative companies. https://t.co/mIQ0qAXiQq'),
 Status(ID=1386760888246546436, ScreenName=adenatfriedman, Created=Mon Apr 26 19:15:35 +0000 2021, Text="It was a pleasure speaking with @ozy's @carloswatson  about how technology and innovation is reshaping the markets, advancing inclusive growth and prosperity through @Nasdaq's Purpose Initiative, the increase in retail trading and much more.\n\nht

In [10]:
def save_timeline(twitter_timeline=None):
    
    for acct in twitter_timeline.keys():
        
        with open("twitter_json/" + acct + ".json", "w+") as f:
            
            for tweets in twitter_timeline[acct]:
                f.write(json.dumps(tweets._json))
                f.write("\n")
                
    return f"{len(twitter_timeline.keys())} CEOs' twitter timeline SAVED"

In [11]:
save_timeline(twitter_timeline)

"178 CEOs' twitter timeline SAVED"

In [12]:
# from Loughran-McDonald Sentiment Word Lists
# link: https://sraf.nd.edu/textual-analysis/resources/#LM%20Sentiment%20Word%20Lists

def get_word_list(csv_file):
    
    words = pd.read_csv(csv_file, sep = ",", header = None)
    words = list(words[0].str.lower())
    return "|".join(words)

In [13]:
import os

dict_words = {}

for csv_file in os.listdir("word_list/"):
    
    dict_words[csv_file[:-4]] = get_word_list("word_list/" + csv_file)
    print(f"Created: {csv_file[:-4]}")

Created: litigious
Created: positive
Created: negative
Created: constraining
Created: uncertainty
Created: weak_modal
Created: financial
Created: strong_modal


In [14]:
dict_words["constraining"]

'abide|abiding|bound|bounded|commit|commitment|commitments|commits|committed|committing|compel|compelled|compelling|compels|comply|compulsion|compulsory|confine|confined|confinement|confines|confining|constrain|constrained|constraining|constrains|constraint|constraints|covenant|covenanted|covenanting|covenants|depend|dependance|dependances|dependant|dependencies|dependent|depending|depends|dictate|dictated|dictates|dictating|directive|directives|earmark|earmarked|earmarking|earmarks|encumber|encumbered|encumbering|encumbers|encumbrance|encumbrances|entail|entailed|entailing|entails|entrench|entrenched|escrow|escrowed|escrows|forbade|forbid|forbidden|forbidding|forbids|impair|impaired|impairing|impairment|impairments|impairs|impose|imposed|imposes|imposing|imposition|impositions|indebted|inhibit|inhibited|inhibiting|inhibits|insist|insisted|insistence|insisting|insists|irrevocable|irrevocably|limit|limiting|limits|mandate|mandated|mandates|mandating|mandatory|manditorily|necessitate|nec

In [15]:
cols = ["account_name", "twitter_date", "twitter_time",
        "twitter_text", "total_count", "constraining",
        "litigious", "negative", "positive",
        "strong_modal", "weak_model", "uncertainty",
        "financial", "following", "follower", "account_created"]

In [ ]:
# option 1: save each CEO's Twitter into Excel files (.xlsx)
# Running Time: 1015.959147021 sec

import re

total_row_count = 0

for json_file in os.listdir("twitter_json/"):
    
    if json_file.endswith(".json"):
        
        with open("twitter_json/" + json_file) as timeline:
            output = pd.DataFrame(columns = cols)
            
            for tweet_info in timeline:
                data = json.loads(tweet_info)
                
                account_name = json_file[:-5]
                tweet_created_at = pd.to_datetime(data["created_at"]).tz_convert(tz = "America/New_York")
                acct_created_at = pd.to_datetime(data["user"]["created_at"]).tz_convert(tz = "America/New_York")
                tweet = data["full_text"]
                
                twitter_date = tweet_created_at.date()
                twitter_time = tweet_created_at.time()
                total_count = len(tweet.split())
                constraining = len(re.findall(dict_words["constraining"], tweet, flags = re.IGNORECASE))
                litigious = len(re.findall(dict_words["litigious"], tweet, flags = re.IGNORECASE))
                negative = len(re.findall(dict_words["negative"], tweet, flags = re.IGNORECASE))
                positive = len(re.findall(dict_words["positive"], tweet, flags = re.IGNORECASE))
                strong_modal = len(re.findall(dict_words["strong_modal"], tweet, flags = re.IGNORECASE))
                weak_model = len(re.findall(dict_words["weak_modal"], tweet, flags = re.IGNORECASE))
                uncertainty = len(re.findall(dict_words["uncertainty"], tweet, flags = re.IGNORECASE))
                financial = len(re.findall(dict_words["financial"], tweet, flags = re.IGNORECASE))
                following = data["user"]["friends_count"]
                follower = data["user"]["followers_count"]
                account_created = acct_created_at.date()
                
                output = output.append({"account_name":account_name,
                                        "twitter_date":twitter_date,
                                        "twitter_time":twitter_time,
                                        "twitter_text":tweet,
                                        "total_count":total_count,
                                        "constraining":constraining,
                                        "litigious":litigious,
                                        "negative":negative,
                                        "positive":positive,
                                        "strong_modal":strong_modal,
                                        "weak_model":weak_model,
                                        "uncertainty":uncertainty,
                                        "financial":financial,
                                        "following":following,
                                        "follower":follower,
                                        "account_created":account_created}, ignore_index = True)
                row_count = len(output.index)
                
                output.to_excel("twitter_excel/" + account_name + ".xlsx", index = False)
                
            print(f"Saved as {json_file[:-5]}.xlsx | Total Rows: {row_count}")
            total_row_count = total_row_count + row_count
            
print(f"Done | Total Rows: {total_row_count}")

In [ ]:
pd.read_excel("twitter_excel/adenatfriedman.xlsx").head(3)

In [16]:
# option 2: save all CEOs' Twitter into an Excel file (.xlsx)
# Running Time: 421.6426406390001 sec

import re

output = pd.DataFrame(columns = cols)

for json_file in os.listdir("twitter_json/"):
    
    if json_file.endswith(".json"):
        
        with open("twitter_json/" + json_file) as timeline:
            
            
            for tweet_info in timeline:
                data = json.loads(tweet_info)
                
                account_name = json_file[:-5]
                tweet_created_at = pd.to_datetime(data["created_at"]).tz_convert(tz = "America/New_York")
                acct_created_at = pd.to_datetime(data["user"]["created_at"]).tz_convert(tz = "America/New_York")
                tweet = data["full_text"]
                
                twitter_date = tweet_created_at.date()
                twitter_time = tweet_created_at.time()
                total_count = len(tweet.split())
                constraining = len(re.findall(dict_words["constraining"], tweet, flags = re.IGNORECASE))
                litigious = len(re.findall(dict_words["litigious"], tweet, flags = re.IGNORECASE))
                negative = len(re.findall(dict_words["negative"], tweet, flags = re.IGNORECASE))
                positive = len(re.findall(dict_words["positive"], tweet, flags = re.IGNORECASE))
                strong_modal = len(re.findall(dict_words["strong_modal"], tweet, flags = re.IGNORECASE))
                weak_model = len(re.findall(dict_words["weak_modal"], tweet, flags = re.IGNORECASE))
                uncertainty = len(re.findall(dict_words["uncertainty"], tweet, flags = re.IGNORECASE))
                financial = len(re.findall(dict_words["financial"], tweet, flags = re.IGNORECASE))
                following = data["user"]["friends_count"]
                follower = data["user"]["followers_count"]
                account_created = acct_created_at.date()
                
                output = output.append({"account_name":account_name,
                                        "twitter_date":twitter_date,
                                        "twitter_time":twitter_time,
                                        "twitter_text":tweet,
                                        "total_count":total_count,
                                        "constraining":constraining,
                                        "litigious":litigious,
                                        "negative":negative,
                                        "positive":positive,
                                        "strong_modal":strong_modal,
                                        "weak_model":weak_model,
                                        "uncertainty":uncertainty,
                                        "financial":financial,
                                        "following":following,
                                        "follower":follower,
                                        "account_created":account_created}, ignore_index = True)
                
            print(f"Done Extracting: {json_file}")
            
output.to_excel("ceo_twitter.xlsx", index = False)

print(f"Done | Total Rows: {len(output.index)}")

Done Extracting: msiegal1.json
Done Extracting: tavetter.json
Done Extracting: lkurzius.json
Done Extracting: glentullman.json
Done Extracting: JDVFab5.json
Done Extracting: devinwenig.json
Done Extracting: kaufer.json
Done Extracting: DrakeEileen.json
Done Extracting: benioff.json
Done Extracting: Walt4NH.json
Done Extracting: asymson.json
Done Extracting: MicronCEO.json
Done Extracting: blipke1.json
Done Extracting: douglebda.json
Done Extracting: mindygrossman.json
Done Extracting: IAmPapaJohn.json
Done Extracting: jeffreyettinger.json
Done Extracting: Dirkvandeput.json
Done Extracting: DRichard_Huston.json
Done Extracting: CEOMikeJackson.json
Done Extracting: gregbrownmoto.json
Done Extracting: jory_marino.json
Done Extracting: larryellison.json
Done Extracting: TechTargetCEO.json
Done Extracting: bclhms.json
Done Extracting: Les_Wexner.json
Done Extracting: howardlance55.json
Done Extracting: frankblake.json
Done Extracting: kentthiry.json
Done Extracting: jeffimmelt.json
Done Ext

In [17]:
ceo_twitter = pd.read_excel("ceo_twitter.xlsx")
print(ceo_twitter.shape)
ceo_twitter.head(3)

(27658, 16)


,account_name,twitter_date,twitter_time,twitter_text,total_count,constraining,litigious,negative,positive,strong_modal,weak_model,uncertainty,financial,following,follower,account_created
0,msiegal1,2021-02-01,08:42:50,@DanRDimicco Alejandro Mayorkas is the most ho...,14,0,0,0,2,0,1,1,0,50,108,2011-03-16
1,msiegal1,2020-06-11,20:35:50,@Isaac_Herzog @zeev_elkin @HaSochnut Noone is ...,6,0,0,0,0,0,0,0,0,50,108,2011-03-16
2,msiegal1,2020-05-05,15:55:52,"With remdesivir, Gilead finds itself at strate...",18,0,0,0,0,0,0,1,0,50,108,2011-03-16
